In [1]:
import sys
import os
from tempfile import mkdtemp
sys.path.insert(0, '..')

import numpy as np
from process_bigraph import Composite
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes import CORE 


Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator']


1. Define inputs

In [2]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')

2. Define composition document (manuscript?)

In [3]:
manuscript = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'results': ['results_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'results': 'tree[string]',
            }
        },
        'inputs': {
            'results': ['results_store'],
        }
    }
}


output_bridge = {
    'inputs': {
        'results': ['results_store']
    },
    'outputs': {
        'results': ['results_store']
    }
}

3. Create composition with doc specifications

In [4]:
composition_config = {'state': manuscript, 'bridge': output_bridge}
composition = Composite(
    config=composition_config,
    core=CORE
)

2024-06-05 16:19:35.113 - amici.sbml_import - INFO - Finished importing SBML                         (4.43E-02s)
2024-06-05 16:19:35.165 - amici.sbml_import - INFO - Finished processing SBML observables            (4.86E-02s)
2024-06-05 16:19:35.170 - amici.sbml_import - INFO - Finished processing SBML event observables      (6.67E-07s)
2024-06-05 16:19:35.195 - amici.de_model - INFO - Finished computing xdot                            (4.45E-03s)
2024-06-05 16:19:35.202 - amici.de_model - INFO - Finished computing x0                              (3.86E-03s)
2024-06-05 16:19:35.223 - amici.de_model - INFO - Finished computing w                               (1.64E-02s)
2024-06-05 16:19:36.002 - amici.de_export - INFO - Finished generating cpp code                      (7.73E-01s)
2024-06-05 16:19:45.494 - amici.de_export - INFO - Finished compiling cpp code                       (9.49E+00s)


In [5]:
dir(composition)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_updates',
 'bridge',
 'bridge_updates',
 'composition',
 'config',
 'config_schema',
 'core',
 'cycle_step_state',
 'determine_steps',
 'edge_paths',
 'emit_port',
 'emitter_paths',
 'expire_process_paths',
 'find_instance_paths',
 'front',
 'gather_results',
 'global_time_precision',
 'initial_state',
 'inputs',
 'interface',
 'invoke',
 'merge',
 'node_dependencies',
 'outputs',
 'process_paths',
 'process_schema',
 'process_update',
 'reset_step_state',
 'run',
 'run_process',
 'run_steps',
 'state',
 'step_dependencies',
 'step_paths',
 'step_triggers',
 'steps_remaining',
 'steps_run',
 'to_ru

In [6]:
composition.state

{'time_store': None,
 'tellurium_floating_species_store': {'PX': None,
  'PY': None,
  'PZ': None,
  'X': None,
  'Y': None,
  'Z': None},
 'tellurium_model_parameters_store': {'eff': 0.0,
  'n': 0.0,
  'KM': 0.0,
  'tau_mRNA': 0.0,
  'tau_prot': 0.0,
  'ps_a': 0.0,
  'ps_0': 0.0,
  'beta': 0.0,
  'alpha0': 0.0,
  'alpha': 0.0,
  't_ave': 0.0,
  'kd_mRNA': 0.0,
  'kd_prot': 0.0,
  'k_tl': 0.0,
  'a_tr': 0.0,
  'a0_tr': 0.0},
 'tellurium_reactions_store': {'Reaction1': 0.0,
  'Reaction2': 0.0,
  'Reaction3': 0.0,
  'Reaction4': 0.0,
  'Reaction5': 0.0,
  'Reaction6': 0.0,
  'Reaction7': 0.0,
  'Reaction8': 0.0,
  'Reaction9': 0.0,
  'Reaction10': 0.0,
  'Reaction11': 0.0,
  'Reaction12': 0.0},
 'amici': {'_type': 'step',
  'address': 'local:utc-amici',
  'config': {'model': {'model_source': '/var/folders/s3/2mt567997bj0_z5_19sd89500000gn/T/tmp9uzih8m2/BIOMD0000000012_url.xml',
    'model_id': '',
    'model_language': 'sbml',
    'model_name': 'composite_process_model',
    'model_chang